In [16]:
from Bio import SeqIO
import numpy as np
import pandas as pd


In [17]:
vocab = SeqIO.to_dict(SeqIO.parse('sequences.fasta', 'fasta'))

uniq_seq = list(map(lambda x: str(x.seq), vocab.values()))
uniq_seq = ''.join(uniq_seq)

uniq_hist = list(map(lambda x: x.split('|')[-2], vocab.keys()))

vocab = list(set(uniq_seq)) + list(set(uniq_hist))

print(vocab)

['A', 'X', 'S', 'B', 'M', 'Q', 'I', '-', 'L', 'T', 'K', 'W', 'E', 'V', 'G', 'F', 'R', 'P', 'C', 'H', 'N', 'Y', 'D', 'H2A', 'H4', 'H1', 'H3', 'H2B']


In [18]:
nums = list(range(29))
dict_1 = dict(zip(vocab, nums))
dict_2 = dict(zip(nums, vocab))

print(dict_1)
print(dict_2)


{'A': 0, 'X': 1, 'S': 2, 'B': 3, 'M': 4, 'Q': 5, 'I': 6, '-': 7, 'L': 8, 'T': 9, 'K': 10, 'W': 11, 'E': 12, 'V': 13, 'G': 14, 'F': 15, 'R': 16, 'P': 17, 'C': 18, 'H': 19, 'N': 20, 'Y': 21, 'D': 22, 'H2A': 23, 'H4': 24, 'H1': 25, 'H3': 26, 'H2B': 27}
{0: 'A', 1: 'X', 2: 'S', 3: 'B', 4: 'M', 5: 'Q', 6: 'I', 7: '-', 8: 'L', 9: 'T', 10: 'K', 11: 'W', 12: 'E', 13: 'V', 14: 'G', 15: 'F', 16: 'R', 17: 'P', 18: 'C', 19: 'H', 20: 'N', 21: 'Y', 22: 'D', 23: 'H2A', 24: 'H4', 25: 'H1', 26: 'H3', 27: 'H2B'}


In [23]:
sequences = SeqIO.to_dict(SeqIO.parse('sequences.fasta', 'fasta'))
sequences_val = list(map(lambda y: str(y.seq), sequences.values()))

seq_tensors = list(map(lambda x: list(map(lambda k: dict_1[k], x)), sequences_val))
hist_tensors = list(map(lambda x: dict_1[x.split('|')[-2]], sequences))
print(len(seq_tensors))
print(len(hist_tensors))
print(hist_tensors[0])

565
565
23


Создайте два словаря: в первом словаре ключом является символ, а значением его порядковый номер в словаре vocab; во втором словаре - наоборот. 

Первый словарь понадобится для того, чтобы закодировать последовательность и целевые значения (тип гистона) в виде тензора. 

Второй словарь, чтобы “превратить” тензор (например, полученный в результате предсказания) с исходный символ или тип гистона. 

Используя первый словарь, получите список тензоров, в котором каждый тензор - это закодированная последовательность, и тензор, хранящий закодированные целевые значения (типы гистонов для каждой последовательности). 

Сколько последовательностей получилось в предобработанном датасете (длина списка тензоров)?

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from sklearn.model_selection import train_test_split

class SequenceDataset(Dataset):
    def __init__(self, sequences, targets):
        self.sequences = sequences
        self.targets = targets
        
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        return self.sequences[idx], self.targets[idx]

embedding_dim = 5
hidden_dim = 9
vocab_size = len(seq_tensors)

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam

seq_tensors = torch.tensor(seq_tensors)
hist_tensors = torch.tensor(hist_tensors)

class RNNWithAttentionModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim=5, hidden_dim=9, random_seed=5):
        super(RNNWithAttentionModel, self).__init__()
        torch.manual_seed(random_seed)
        torch.cuda.manual_seed(random_seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        
        # Embedding layer
        self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)
        
        # RNN layer (LSTM)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        
        # Attention layer
        self.attention = nn.Linear(hidden_dim, 1)
        
        # Output layer
        self.fc = nn.Linear(hidden_dim, vocab_size)
    
    def forward(self, x):
        # Embed the input
        x = self.embedding(x)  
        
        # RNN processing
        rnn_out, _ = self.rnn(x)  
        
        # Compute attention scores
        attention_scores = self.attention(rnn_out)
        attention_weights = F.softmax(attention_scores, dim=1)
        
        # Compute context vector (weighted sum)
        context = torch.sum(attention_weights * rnn_out, dim=1)
        
        # Final output
        out = self.fc(context)
        
        return out

# Пример использования:
test_sequence = [1, 5, 10, 15]  # или строка "ACGT"
pred_class, class_probs = predict_single(model, test_sequence, vocab)
print(f"Предсказанный класс: {pred_class}")
print(f"Вероятности классов: {class_probs}")


# Training setup
vocab_size = len(seq_tensors)
model = RNNWithAttentionModel(vocab_size, embedding_dim, hidden_dim)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.01)

# Training loop
def train_model(model, train_loader, epochs=700):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch_idx, (data, target) in enumerate(train_loader):
        # for i in range(len(X_train)) :
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        if (epoch+1) % 50 == 0:
            print(f'Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}')

# Example usage (assuming you have DataLoader prepared)
# train_loader = ...  # Your DataLoader
# train_model(model, train_loader)
X_train, X_test, y_train, y_test = train_test_split(seq_tensors, hist_tensors, test_size=0.2, random_state=5) 

batch_s = 32

train_ds = SequenceDataset(X_train, y_train)
val_ds = SequenceDataset(X_test, y_test)

train_l = DataLoader(train_ds, batch_size=batch_s, shuffle=True, num_workers=2)
val_l = DataLoader(val_ds, batch_size=batch_s, shuffle=True, num_workers=2)

train_model(model, train_l)

/tmp/ipykernel_3776/3981069463.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  seq_tensors = torch.tensor(seq_tensors)
/tmp/ipykernel_3776/3981069463.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  hist_tensors = torch.tensor(hist_tensors)


Epoch 50, Loss: 1.2587
Epoch 100, Loss: 0.3835
Epoch 150, Loss: 0.1898
Epoch 200, Loss: 0.0712
Epoch 250, Loss: 0.0827
Epoch 300, Loss: 0.0410
Epoch 350, Loss: 0.0241
Epoch 400, Loss: 0.0331
Epoch 450, Loss: 0.0457
Epoch 500, Loss: 0.0218
Epoch 550, Loss: 0.0133
Epoch 600, Loss: 0.0105
Epoch 650, Loss: 0.0049
Epoch 700, Loss: 0.0027


Для выполнения задания вы можете самостоятельно создать Python-ноутбук, а в LMS прикладывать лишь требуемые результаты. В этом задании вам необходимо написать рекуррентную нейросеть с механизмом внимания с использованием PyTorch. В качестве входного слоя создайте Embedding. Далее создайте 1 рекуррентный слой. В качестве слоя внимания используйте линейную трансформацию. В качестве выходного слоя используйте линейную трансформацию. Используйте код для решения данной задачи:

Запустите обучение нейросети на 700 эпохах. Используйте значение кросс-энтропии в качестве функции потерь и адаптивный алгоритм Adam со скоростью обучения 0.01 в качестве алгоритма оптимизации. Выберете верно утверждение:

Используя обученную модель нейросети предскажите тип гистона для последовательности: 

seq_fasta = '''>Pan|XP_003311177.1|HTYPE|HVARIANT 

MSGRGKQGGKARTKAKTRSSRAGLQFPVGRVHRLLRKGNYAERVGAGAPVYLAAVLEYLT 

AEILELAGNAARDNKKTRIIPRHLQLAIRNDEELNKLLGKVTIAQGGVLPNIQAVLLPKK 

TESHHKAKGK'''

In [31]:
def predict_single(model, sequence, vocab, max_len=50, device='cpu'):
    """
    Предсказание для одной последовательности
    
    Args:
        model: обученная модель
        sequence: входная последовательность (список индексов)
        vocab: словарь для преобразования токенов
        max_len: максимальная длина последовательности
        device: устройство для вычислений
    
    Returns:
        tuple: (предсказанный класс, вероятности классов)
    """
    # Преобразуем последовательность в тензор
    if isinstance(sequence, str):
        # Если на входе строка, токенизируем ее
        sequence = [vocab.get(token, 0) for token in sequence]  # 0 для OOV
    
    # Добавляем padding
    padded_seq = torch.nn.functional.pad(
        torch.tensor(sequence),
        (0, max_len - len(sequence)),
        value=0
    )
    
    # Добавляем batch dimension
    input_tensor = padded_seq.unsqueeze(0).to(device)
    
    # Делаем предсказание
    model.eval()
    with torch.no_grad():
        output = model(input_tensor)
        probs = torch.softmax(output, dim=1)
        pred = torch.argmax(probs, dim=1).item()
    
    return pred, probs.squeeze().cpu().numpy()

# Пример использования:
test_sequence = 'MSGRGKQGGKARTKAKTRSSRAGLQFPVGRVHRLLRKGNYAERVGAGAPVYLAAVLEYLTAEILELAGNAARDNKKTRIIPRHLQLAIRNDEELNKLLGKVTIAQGGVLPNIQAVLLPKKTESHHKAKGK'  # или строка "ACGT"
pred_class, class_probs = predict_single(model, test_sequence, dict_2)
print(f"Предсказанный класс: {pred_class}")
print(f"Вероятности классов: {class_probs}")

Предсказанный класс: 25
Вероятности классов: [3.6703043e-09 2.9734539e-09 2.7026759e-09 3.3443814e-09 3.6848591e-09
 1.7214317e-09 2.2935387e-09 1.7233273e-09 2.0864197e-09 1.5227409e-09
 2.4711433e-09 1.8455484e-09 3.7518291e-09 2.3855673e-09 3.2569794e-09
 2.1371651e-09 2.8205573e-09 1.8180104e-09 2.0338393e-09 1.4317328e-09
 3.5843315e-09 2.3692881e-09 2.0004218e-09 1.9196827e-06 1.1646406e-10
 9.9994230e-01 1.6399394e-05 3.8153339e-05 4.1861141e-09 3.3393650e-09
 2.5912874e-09 1.6337061e-09 1.7432090e-09 2.5053803e-09 3.1800571e-09
 2.9284661e-09 1.6818146e-09 1.8076030e-09 5.4480296e-09 2.3044573e-09
 3.0361917e-09 2.6623332e-09 2.1679945e-09 1.8101768e-09 1.6033944e-09
 3.3101628e-09 2.0156885e-09 1.9749793e-09 4.3199844e-09 2.8158540e-09
 1.9166084e-09 1.8296501e-09 1.9162283e-09 3.0509482e-09 2.3431683e-09
 2.5632483e-09 1.2803018e-09 2.5669273e-09 1.8733926e-09 5.3419420e-09
 2.3240203e-09 3.8736871e-09 2.3721098e-09 2.0972601e-09 1.2146811e-09
 3.3580474e-09 2.3361890e-09 1.2